<div align="Center"><h1>Data Manipulation</h1></div>

In order to implement the data model previously discussed, and perform visualization to gain insights into this data set, I'm using the Python DataFrame framework to manipulate the data and create necessary calculated measures/dimensions. This method can be applied to a live data connection, so it would work well even if we wanted to analyze data directly from SFDC/Workday/other systems.

<b>1. Data Import</b>

Below, I will create a data view (reporting_view) by joining the provided Bookings and Roster tables.

In [1]:
import pandas as pd

bookings=pd.read_csv('C:/Users/nquan/Google Drive/Dropbox/Jobs/Medallia/Data Challenge/Booked by FY - SFDC.csv')
roster=pd.read_csv('C:/Users/nquan/Google Drive/Dropbox/Jobs/Medallia/Data Challenge/Roster.csv', keep_default_na=False)

In order to join Bookings and Roster, we need a common primary key. Recognizing that Opportunity Owner is unique and can be used as Bookings' PK, I will generate Employee Key from Employee Full Name, to be used as PK for Roster.

In [2]:
employee_key=pd.DataFrame(roster['Employee Full Name'].str.replace(r' (?![a-zA-Z])',''))
employee_key.columns=['Opportunity Owner']
roster=pd.concat([roster, employee_key], axis=1)
reporting_view=pd.merge(roster, bookings, on=['Opportunity Owner'], how='left')

<b>2. Creating the Fiscal Calendar hierarchy</b>

Now that we have reporting_view initialized, the next step is to apply the data model for Fiscal Hierarchies. Using business rules provided, Fiscal Calendar table is created with the following attributes:
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-yw4l">Calendar Date</th>
    <th class="tg-yw4l">Fiscal Year</th>
    <th class="tg-yw4l">Fiscal Quarter</th>
    <th class="tg-yw4l">Fiscal Month</th>
  </tr>
  <tr>
    <td class="tg-yw4l"></td>
    <td class="tg-yw4l"></td>
    <td class="tg-yw4l"></td>
    <td class="tg-yw4l"></td>
  </tr>
</table>

This table is then joined with Reporting View to add Fiscal dimensions for Booking Date and Close Date:

In [3]:
fiscal=pd.read_csv('C:/Users/nquan/Google Drive/Dropbox/Jobs/Medallia/Data Challenge/Fiscal Calendar.csv')

fiscal.columns=['Booking Date', 'Booking Fiscal Year', 'Booking Fiscal Quarter', 'Booking Fiscal Month']
reporting_view=pd.merge(reporting_view, fiscal, on=['Booking Date'], how='left')

fiscal.columns=['Close Date', 'Close Fiscal Year', 'Close Fiscal Quarter', 'Close Fiscal Month']
reporting_view=pd.merge(reporting_view, fiscal, on=['Close Date'], how='left')

reporting_view=reporting_view.drop_duplicates()

<b>3. Calculating measures/dimensions for visual analysis</b>

In order to answer the questions posed in this challenge, a number of new attributes need to be calculated. Attainment is measured by Cumulative swNACV over Cumulative Quota. Quota is based on Tenure. Tenure is based on Start and End Date (or current date, which is assumed to be 5/1/2017). The attributes are calculated as follow:

In [4]:
#Extract a sub-view with core attributes needed for calculations
view_extract=reporting_view.groupby(['Employee Full Name', 'Start Date', 'End Date'])['swNACV'].sum().to_frame().reset_index()
view_extract.columns=['Employee Full Name', 'Start Date', 'End Date', 'Total swNACV']

#Calculate Tenure
from datetime import date
from dateutil import relativedelta
from dateutil import parser

tenure=[]
startdatelist=list(view_extract['Start Date'])
enddatelist=list(view_extract['End Date'])
for i in range(0,len(view_extract)):
    startdate=parser.parse(startdatelist[i])
    if enddatelist[i]=='':
        enddate=parser.parse('5/1/2017')
    else:
        enddate=parser.parse(enddatelist[i])
    difference=relativedelta.relativedelta(enddate, startdate)
    tenure.append(difference.years*12+difference.months)

tenuredf=pd.DataFrame(tenure, columns=['Tenure'], index=view_extract.index)

#Calculate Cumulative Quota
cumulative_quota=[]
for i in range(0,len(view_extract)):
    if tenure[i]>5:
        cumulative_quota.append((tenure[i]-5)*58333.33)
    else:
        cumulative_quota.append(0)
quotadf=pd.DataFrame(cumulative_quota, columns=['Cumulative Quota'], index=view_extract.index)

#Calculate Attainment
totalswNACVlist=list(view_extract['Total swNACV'])
attainment=[]
for i in range(0,len(view_extract)):
    if totalswNACVlist[i]!=totalswNACVlist[i]:
        attainment.append(0)
    else:
        if cumulative_quota[i]==0:
            attainment.append(1)
        else:
            attainment.append(totalswNACVlist[i]/cumulative_quota[i])
attainmentdf=pd.DataFrame(attainment, columns=['Attainment'], index=view_extract.index)

#Add new attributes to reporting_view
view_extract=pd.concat([view_extract, tenuredf, quotadf, attainmentdf], axis=1)
view_extract=view_extract.drop(['Start Date', 'End Date'], axis=1)
reporting_view=pd.merge(reporting_view, view_extract, on=['Employee Full Name'], how='left')

<b>4. Data export/Connecting to Tableau</b>

reporting_view is now ready for visualization and analysis in Tableau. I will write it to a .csv, then connect it to Tableau to start the visual analysis.

In [5]:
reporting_view.to_csv(path_or_buf='C:/Users/nquan/Google Drive/Dropbox/Jobs/Medallia/Data Challenge/Reporting View.csv', index=False)